# Road Following - Live demo(TensorRTデモ)
Pytorchで学習したモデルをTensorRTモデルに変換したことで高速処理が可能になりました。  
このノートブックでは、TensorRT化したモデルを使うことでカクツキを抑えてJetRacerがなめらかに走行することを確認できます。

### Load Trained Model(学習済みモデルの読み込み)
``convert_to_trt.ipynb``ノートブックの指示に従って、すでに``road_following_model_trt.pth``を作成していることを想定します。
TensorRTモデル``road_following_model_trt.pth``を読み込みます。

In [ ]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('road_following_model_trt.pth'))

車両制御クラスをインスタンス化します。

In [ ]:
from jetracer.nvidia_racecar import NvidiaRacecar
car = NvidiaRacecar()

一度カメラ用のデーモンを再起動しておきます。

In [ ]:
!echo jetson | sudo -S systemctl restart nvargus-daemon

カメラクラスをインスタンス化します。

In [ ]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=40)

次に、車体固有のパラメータを設定します。  
``basic_motion.ipynb``で確認したパラメータを設定します。

|パラメータ|機能|値範囲|解説|
|:--|:--|:--|:--|
|car.steering|左右ステアリング値|[-1.0, 1.0]|現在のステアリングの値。0.0がニュートラル位置(理論上まっすぐ進む状態。実際は車体のがたつき、ゆがみ等でまっすぐ進まないことが多いです)。|
|car.steering_gain|ステアリング適用率|[-1.0, 1.0]|TT-02の場合は**0.3**付近がちょうどいい値です。ステアリング値をそのまま適用すると、物理的にそれ以上左右に切れない状態になります。その状態ではサーボを痛めてしまうため、この値を小さくして**car.steering**が[-1.0, 1.0]の範囲でサーボから異音がしない値に設定します。実際に適用されるステアリング値は(car.steering * car.steering_gain + car.steering_offset)となります。**car.steering_gain**の値がプラスかマイナスかは、車種毎に固定になります（サーボの取り付け向きで決まります）。調整方法としては、まずは**car.steering_offset**を調整して、**car.steering = 0**で車体がまっすぐ走行できるようにします。次に**car.steering_gain**を小さい値から設定します。ステアリング値が-1.0, 1.0の時にサーボから異音がしないようにこの値を上げていき、左右どちらかで異音が始まるぎりぎりの値を設定します。反対側にもっと切れる状態であっても、ここで止めます。|
|car.steering_offset|ステアリングニュートラル補正値|[-1.0, 1.0]|車体がまっすぐ走行する位置に設定します。TT-02ノーマル車体の場合はステアリングのがたつきが大きく、完全にまっすぐ走行させることは不可能ですので、だいたいまっすぐ走行できればOKです。|
|car.steering_channel|ステアリング用のPCA9685出力PIN番号|[0, 15]|PCA9685は16個のチャンネルを持ちますが、JetRacerのステアリング(サーボ制御)はPIN 0を使います。|
|car.throttle|前後スロットル値|[-1.0, 1.0]|現在のスロットルの値。0.0がニュートラル位置。|
|car.throttle_gain|スロットル適用率|[-1.0, 1.0]|TT-02の場合は**-0.69**にしておくことで、車体の最大パフォーマンスを得られます(最大前進時のPWM信号のHIGH時間が1100μsになる値)。この状態で**car.throttle = 0.13**くらいで車両が走行しはじめます(バッテリー電圧や重量、回転抵抗にもよります)。プラスかマイナスかはESC毎に固定です。**car.throttle**がプラスの時に前進するように**car.throttle_gain**のプラスマイナスを決めます。|
|car.throttle_offset|スロットルニュートラル補正値|[-1.0, 1.0]|何もしないときに車体が停止する値に設定します。|
|car.throttle_channel|スロットル用のPCA9685出力PIN番号|[0, 15]|PCA9685は16個のチャンネルを持ちますが、JetRacerのスロットル(ESC制御)はPIN 1を使います。|

In [ ]:
# 車両パラメータを初期化します
car.steering = 0
car.steering_gain = 0.3
car.steering_offset = 0
car.steering_channel = 0
car.throttle = 0
car.throttle_gain = -0.69
car.throttle_offset = 0
car.throttle_channel = 1

# 走行しよう！
次のコードを実行すると、車両の走行準備が整います。  
最初は車両を持ち上げた状態で動作を確認するのがいいでしょう。  
準備ができたら、実際にJetRacerをコース上において自動走行してみましょう。

In [ ]:
import ipywidgets.widgets as widgets
from IPython.display import display
from jetracer.utils import preprocess
import numpy as np
import threading
import traitlets
import cv2
import time
import ipywidgets
from ipywidgets import Label
from jetcam.utils import bgr8_to_jpeg

# Controll Widgets
style = {'description_width': 'initial'}
steering_slider = widgets.FloatSlider(description='steering', style=style, min=-1.0, max=1.0, step=0.01, orientation='horizontal')
steering_gain = widgets.BoundedFloatText(description='steering_gain', style=style ,min=-1.0, max=1.0, step=0.01, value=car.steering_gain)
steering_offset = widgets.BoundedFloatText(description='steering_offset', style=style, min=-1.0, max=1.0, step=0.01, value=car.steering_offset)
throttle_slider = widgets.FloatSlider(description='throttle', style=style, min=-1.0, max=1.0, step=0.01, orientation='vertical')
throttle_gain = widgets.BoundedFloatText(description='throttle_gain', style=style, min=-1.0, max=1.0, step=0.01, value=car.throttle_gain)
throttle_offset = widgets.BoundedFloatText(description='throttle_offset', style=style, min=-1.0, max=1.0, step=0.01, value=car.throttle_offset)

# create a horizontal box container to place the sliders next to eachother
slider_container = widgets.HBox([throttle_slider, steering_slider])
slider_container.layout.align_items='center'
value_container = widgets.VBox([steering_gain, steering_offset, throttle_gain, throttle_offset])

control_container = widgets.HBox([slider_container, value_container])
control_container.layout.align_items='center'

# Link
steering_link = traitlets.link((steering_slider, 'value'), (car, 'steering'))
steering_gain_link = traitlets.link((steering_gain, 'value'), (car, 'steering_gain'))
steering_offset_link = traitlets.link((steering_offset, 'value'), (car, 'steering_offset'))
throttle_link = traitlets.link((throttle_slider, 'value'), (car, 'throttle'))
throttle_gain_link = traitlets.link((throttle_gain, 'value'), (car, 'throttle_gain'))
throttle_offset_link = traitlets.link((throttle_offset, 'value'), (car, 'throttle_offset'))

description_style = {'description_width': 'initial'}

run_button = widgets.Button(description='推論開始')
stop_button = widgets.Button(description='推論停止')
log_widget = widgets.Textarea(description='ログ', style=description_style)

prediction_widget = ipywidgets.Image(format='jpeg', width=camera.width, height=camera.height)

def live():
    global running, count, prediction_widget
    log_widget.value = "live"
    count = 0
    while running:
        count = count + 1
        log_widget.value = str(count) + "回目の推論"
        image = camera.read()
        prediction = image.copy()
        image = preprocess(image).half()
        output = model_trt(image).detach().cpu().numpy().flatten()
        x_pred = float(output[0])
        y_pred = float(output[1])
        car.steering = x_pred
        #car.throttle = y_pred/2.0 + 0.5

        """推論結果を画面に表示したい場合はここを有効化して、最後にdisplay(prediction_widget)でwidgetを表示させます。
        x_ratio = x_pred/2 + 0.5
        y_ratio = y_pred/2 + 0.5
        x = int(camera.width * x_ratio)
        y = int(camera.height * y_ratio)

        prediction = cv2.circle(prediction, (int(x), int(y)), 8, (255, 0, 0), 3)
        prediction_widget.value = bgr8_to_jpeg(prediction)
        """


def run(c):
    global running, execute_thread, start_time
    log_widget.value = "run"
    running = True
    execute_thread = threading.Thread(target=live)
    execute_thread.start()
    start_time = time.time()

def stop(c):
    global running, execute_thread, start_time, count
    end_time = time.time() - start_time
    fps = count/int(end_time)
    log_widget.value = "FPS: " + str(fps) + "(1秒あたりの推論実行回数)"
    running = False
    execute_thread.stop()
    car.throttle = 0
    
run_button.on_click(run)
stop_button.on_click(stop)

# create a horizontal box container to place the sliders next to eachother
run_widget = widgets.VBox([
    widgets.HBox([run_button, stop_button]),
    log_widget
])

# display the container in this cell's output
#display(prediction_widget)
display(Label('スロットルは手動で設定します。ステアリングは推論中は自動設定されます。'))
display(control_container)
display(run_widget)